In [1]:
import subprocess
def runprint(cmd):
    print(subprocess.check_output(cmd.split()).decode())

# Transport model (`lumia.obsoperator` modules and *scripts/lagrange_mp.py* script)

The role of the observation operator in the inversion is to compute the observed values ($y$) corresponding to a given control vector $\mathbf{x}$ (i.e. the vector of variables adjustable by the inversion algorithm). In our case, the observations are a set of atmospheric CO$_2$ concentrations, and the control vector is composed of surface CO$_2$ fluxes, at a variable spatial and temporal resolution, and covering only the *biosphere* (NEE) flux category. The observation operator performs the following tasks:
1. Generate a set of high resolution CO$_2$ fluxes, based on the control vector;
2. Add to it the fluxes from non-optimized categories (fossil, oceans);
3. Compute the impact of these fluxes on CO$_2$ concentrations;
4. Add to it the impact of fluxes outside the regional domain (i.e. background fluxes).

In adjoint mode, the steps are inverted:
1. Compute an adjoint flux field based on a set of model-data mismatches;
2. Compute an adjoint control vector based on this adjoint flux field.

Technically, step 3 and 4 (and 1 of the adjoint) are performed by a transport model (*lagrange_mp.py* script), which is launched as a subprocess by lumia. 
This README focuses on these two steps. The construction of transport model inputs from the control vector is described in the *README_interfaces.ipynb* notebook.

Our transport model relies on pre-computed observation footprints (i.e. arrays storing the sensitivity of observations to individual flux components):

$y^o_{m} = y^o_{bg} + \sum_c\left(\sum_j K^o_{i,j,t} f^c_j\right)$

with $y^o_m$ the model estimate for a given observation $o$; $y^o_{bg}$ is the (prescribed) background concentration (i.e. transport to the observation site of concentrations from the edge of the domain); $K^o$ is the footprint of the observation $o$; $f^c_{i,j,t}$ is the flux estimate for the category $c$ at the coordinates ($i, j, t$).
- the footprints $K$ are pre-computed (using a Lagrangian transport model such as FLEXPART or STILT);
- the background concentrations are also pre-computed (using a global atmospheric transport model such as TM5);
- the fluxes are generated by lumia.

In adjoint mode, the model performs the following calculation:

$f_{c,i,j,t}^{adj} = \sum_o K^o_{i,j,t} dy^o$

with $dy^o$ the model mismatch computed for the observation $o$ in the preceding forward run.

The actual calculations are performed by the *lagrange_mp.py* script, the `lumia.obsoperator` module is in charge of preparing the input files for it, of executing it as a subprocess and of reading in the results. The *lagrange_mp.py* script can therefore be called independently of lumia:

In [4]:
runprint('../transport/lagrange_mp.py --help')

usage: lagrange_mp.py [-h] [--forward] [--adjoint] [--serial]
                      [--checkfile CHECKFILE] [--rc RC] --db DB --emis EMIS
                      [--verbosity VERBOSITY]
                      ...

positional arguments:
  args

optional arguments:
  -h, --help            show this help message and exit
  --forward, -f         Do a forward run
  --adjoint, -a         Do an adjoint run
  --serial, -s          Run on a single CPU
  --checkfile CHECKFILE, -c CHECKFILE
  --rc RC
  --db DB
  --emis EMIS
  --verbosity VERBOSITY, -v VERBOSITY



The `--rc`, `--emis` and `--db` arguments define the path of the input files. The `--serial` argument determines whether the run needs to be parallelized or not. The optional `--verbosity` can be set to `DEBUG` for more messages. The optional `--checkfile` points to a (non-existing) empty file that can be written at the end of the forward run, to ensure that it finished correctly.
​

## Observations/departures file (`--db` argument)

The observations are provided as a tar archive of a `lumia.obsdb` database (see *README_obsdb.py*). The "observations" table must contain the following columns:
- **time**: the time of the observations
- **footprint**: the path to the file storing the footprint corresponding to each observation (see below)
- **background** (in forward runs only): the background concentration for each obs.
- **dy** (in adjoint runs only): the model-data mismatches computed in the forward step.

## Flux file (`--emis` argument)

The fluxes are to be provided as a single netCDF file, with one group for each flux category. Below is the (partial) ncdump header of a flux file:
```
dimensions:
	time_components = 6 ;
	nt = 2920 ;
	nlat = 80 ;
	nlon = 100 ;

group: fossil {
  variables:
  	double emis(nt, nlat, nlon) ;
  		emis:units = "micromol/m2/s" ;
  	double area_m2(nlat, nlon) ;
  	int times_start(nt, time_components) ;
  	int times_end(nt, time_components) ;
  	float lats(nlat) ;
  	float lons(nlon) ;
  } // group fossil

group: ocean {
  variables:
  	double emis(nt, nlat, nlon) ;
  		emis:units = "micromol/m2/s" ;
  	double area_m2(nlat, nlon) ;
  	int times_start(nt, time_components) ;
  	int times_end(nt, time_components) ;
  	float lats(nlat) ;
  	float lons(nlon) ;
  } // group ocean
```
The flux file has two categories: *fossil* and *ocean*. For each flux category, there are 5 mandatory variables:
- emis: the surface flux itself (in $\mu$mol/m$^2$/s);
- lats/lons: the coordinates of the center of the grid points;
- times_start/times_end: the start and end of each time interval.

In adjoint mode, the `--emis` argument specifies the name where the adjoint fluxes are written (in the exact same format).

## Footprint files

The footprints are stored in HDF5 files. Each file can contain multiple footprints. The footprint file **scripts/example_data/nor.100m.2017-01.h5** is provided as an example:

In [5]:
# The file contains many footprints, we just show a fraction of it..
runprint('h5dump -d latitudes -d longitudes -g 20170131140000/20170131150000_20170131120000 -g 20170131140000/20170131120000_20170131090000 footprints/nor.100m.2017-01.h5')

HDF5 "footprints/nor.100m.2017-01.h5" {
DATASET "latitudes" {
   DATATYPE  H5T_IEEE_F64LE
   DATASPACE  SIMPLE { ( 32 ) / ( 32 ) }
   DATA {
   (0): 54.25, 54.75, 55.25, 55.75, 56.25, 56.75, 57.25, 57.75, 58.25, 58.75,
   (10): 59.25, 59.75, 60.25, 60.75, 61.25, 61.75, 62.25, 62.75, 63.25,
   (19): 63.75, 64.25, 64.75, 65.25, 65.75, 66.25, 66.75, 67.25, 67.75,
   (28): 68.25, 68.75, 69.25, 69.75
   }
}
DATASET "longitudes" {
   DATATYPE  H5T_IEEE_F64LE
   DATASPACE  SIMPLE { ( 60 ) / ( 60 ) }
   DATA {
   (0): 0.25, 0.75, 1.25, 1.75, 2.25, 2.75, 3.25, 3.75, 4.25, 4.75, 5.25,
   (11): 5.75, 6.25, 6.75, 7.25, 7.75, 8.25, 8.75, 9.25, 9.75, 10.25, 10.75,
   (22): 11.25, 11.75, 12.25, 12.75, 13.25, 13.75, 14.25, 14.75, 15.25,
   (31): 15.75, 16.25, 16.75, 17.25, 17.75, 18.25, 18.75, 19.25, 19.75,
   (40): 20.25, 20.75, 21.25, 21.75, 22.25, 22.75, 23.25, 23.75, 24.25,
   (49): 24.75, 25.25, 25.75, 26.25, 26.75, 27.25, 27.75, 28.25, 28.75,
   (58): 29.25, 29.75
   }
}
GROUP "20170131140000/20

The file has a hierachical structure:
- The higher level contains two datasets (`latitudes` and `longitudes`, showing the coordinates of the center of the grid points), and many groups, each containing the footprint of one observation (i.e. $K^i$). In the example above, an extract of the group `20170131140000` are displayed. The observations are identified by their time (here 2017/01/31 14:00), which means that there cannot be two observations at the same time in the same file (typically, using different footprint files for different observation sites avoids problems).
- Each footprint group contains itself several (many) subgroups. Each contains the information needed to reconstruct a slice of the footprint along its time axis (i.e. $K^i_t$). The group name corresponds to the time interval (which itself must match the time intervals of the emissions!). For instance the group `20170131140000/20170131120000_20170131090000` contains the sensitivity of the observation `20170131140000` to surface fluxes on 2017/01/13 from 9:00 to 12:00.
    - the `resp` datasets contain the non-zero values of $K$
    - the `ilats` and `ilons` datasets contain the latitude and longitude of the values in the `resp` datasets.
    
The script below shows how to reconstruct the 3D footprints from the example file:

In [6]:
from h5py import File
from datetime import datetime
from numpy import zeros, array
footprints = {}
with File('footprints/nor.100m.2017-01.h5', 'r') as fp:
    lons = fp['longitudes'][:]
    lats = fp['latitudes'][:]
    nlon, nlat = len(lons), len(lats)
    
    # Loop over all the footprints in the file
    print("The file has footprints for the following observations times:")
    for obs in fp.keys():
        if obs not in ['latitudes', 'longitudes']:
            date = datetime.strptime(obs, '%Y%m%d%H%M%S')
            print(date)
            ts, te, fpt = [], [], []
            
            # Loop over all the time steps of the current footprint
            for tstep in sorted(fp[obs]):
                t1, t0 = tstep.split('_')
                ts.append(datetime.strptime(t0, '%Y%m%d%H%M%S'))
                te.append(datetime.strptime(t1, '%Y%m%d%H%M%S'))
                data = zeros((nlat, nlon))
                ilons = fp[obs][tstep]['ilons'][:]
                ilats = fp[obs][tstep]['ilats'][:]
                data[ilats, ilons] = fp[obs][tstep]['resp'][:]
                fpt.append(data)
                
            # Each element of the "footprints" dictionary contains a sub-dictionary, with one "value" variable (the 3D footprint), and four coordinate variables
            footprints[date] = {'start':array(ts), 'end':array(te), 'value':array(fpt), 'lons':lons, 'lats':lats}

The file has footprints for the following observations times:
2017-01-31 11:00:00
2017-01-31 12:00:00
2017-01-31 13:00:00
2017-01-31 14:00:00


## Rc-file (`--rc` argument)

In forward runs, the rc file requires the two following keys:
- `model.transport.split`: determines on how many CPUs the computations are parallelized (see parallelisation section below)
- `path.run`: determines where the model output is written

In adjoint runs, the following keys are also required, to construct the adjoint flux file. These are given here for reference but are normally automatically generated by lumia.
- `emissions.categories`: the names of the flux categories transported
- for each category `cat`, an `emissions.cat.optimize` key (with a boolean value)
- for each category `cat`, an `emissions.cat.interval` key, setting the time interval at which the fluxes are defined
- `time.start` and `time.end`: start and end times of the inversion (in a "yyyy, mm, dd, [HH, mm]" format)

## Parallelisation

The transport model is parallelised by splitting the observation database in chunks of approximately equal size. Several subprocesses are then launched (one for each chunk of the database). This is handled directly by *lagrange_mp.py* script: it splits the database and lauches subprocesses of itself with the `--serial` argument (which specifies that no further splitting should occur).

This parallelization is very efficient as each process reads different footprint files (and the I/O is the limitation here). It has two practical consequences:
- no additional library is required, it can run on any system
- multi-nodes systems are not handled: the number of processes is limited by the number of CPUs on the machine